In [11]:
import re
import pandas as pd
import numpy as np


In [12]:
DNA_AA_map = {"TTT":"F", "TTC":"F", "TTA":"L", "TTG":"L",
              "TCT":"S", "TCC":"S", "TCA":"S", "TCG":"S",
              "TAT":"Y", "TAC":"Y", "TAA":".", "TAG":".",
              "TGT":"C", "TGC":"C", "TGA":".", "TGG":"W",
              "CTT":"L", "CTC":"L", "CTA":"L", "CTG":"L",
              "CCT":"P", "CCC":"P", "CCA":"P", "CCG":"P",
              "CAT":"H", "CAC":"H", "CAA":"Q", "CAG":"Q",
              "CGT":"R", "CGC":"R", "CGA":"R", "CGG":"R",
              "ATT":"I", "ATC":"I", "ATA":"I", "ATG":"M",
              "ACT":"T", "ACC":"T", "ACA":"T", "ACG":"T",
              "AAT":"N", "AAC":"N", "AAA":"K", "AAG":"K",
              "AGT":"S", "AGC":"S", "AGA":"R", "AGG":"R",
              "GTT":"V", "GTC":"V", "GTA":"V", "GTG":"V",
              "GCT":"A", "GCC":"A", "GCA":"A", "GCG":"A",
              "GAT":"D", "GAC":"D", "GAA":"E", "GAG":"E",
              "GGT":"G", "GGC":"G", "GGA":"G", "GGG":"G", }

base_editing_key = {"CBE": ["C", "T"], 
                      "ABE": ["A", "G"]
                     }

bases = 'ACGT'
complements = {'A':'T', 'T':'A', 'G':'C', 'C':'G', 
                   'a':'t', 't':'a', 'g':'c', 'c':'g'}
cas_key = {'Sp': 'NGG', 'SpG': 'NGN', 'SpRY': 'NNN'}

def DNA_to_AA(seq): 
    aa_seq = ''
    for i in range(len(seq)//3): 
        codon = seq[(i*3):(i*3)+3].replace("T", "U")
        aa_seq += DNA_AA_map[codon]
    return aa_seq

def rev_complement(complements, seq): 
    compl = ''
    for i in range(len(seq)): 
        compl += complements[seq[i]]
    return compl[::-1]

def complement(complements, seq): 
    compl = ''
    for i in range(len(seq)): 
        compl += complements[seq[i]]
    return compl


In [13]:
class Gene_BaseEditing(): 
    
    def __init__(self, filepath, output_dir=''): 
        f = open(filepath, "r")
        self.file_content = f.read()
        self.exons_extra, self.exons = self.parse_exons()
    
    # function to read in a .fasta file as its exons, separating into just exon and exon +- 20 bps
    # outputs: list of exons, list of exons +- 20 bps
    def parse_exons(self): 
        exons_extra = []
        i = -1
        for line in self.file_content.split('\n'): 
            if len(line) > 0 and line[0] == '>': 
                exons_extra.append('')
                i += 1
            else: 
                exons_extra[i] += line
        exons = []
        for exon in exons_extra: 
            exons.append(''.join([base for base in exon if base.isupper()]))
        return exons_extra, exons
    
    # generates all fwd and rev potential guides and their metadata
    # while this is computationally naive, genes are usually not that long
    # pseudo outputs: guide sequence, the index of the first bp, the frame of the first bp, the exon # of guide
    def find_all_guides(self, n): 
        self.fwd_guides = []
        prev_frame = 0
        prev_ind = 0
        for e, exon_extra in enumerate(self.exons_extra): 
            for i in range(len(exon_extra)-22): 
                frame = (i+prev_frame-20)%3
                ind = i-20+prev_ind
                self.fwd_guides.append([exon_extra[i:i+23], frame, ind, e])
            prev_frame = (prev_frame+len(exon_extra)-40)%3
            prev_ind += len(exon_extra)-40
            
        self.rev_guides = [[rev_complement(complements, 
                                           g[0])] + [(g[1]+1)%3] + [g[2]+22] + [g[3]] for g in self.fwd_guides]
        

In [14]:
def find_gRNAs(gene_object, mode, cas_type, target_codons=[], window=[4,8], PAM=None): 
    # mode: can be CBE or ABE, or just a list of 2 base edits ex ["C", "T"]
    # cas_type: Sp, SpG, SpRY
    # target_codons: list of codons that we want to make with our base edit
    # window: 4th to 8th bases inclusive by default, can be changed
    # PAM: optional field to input a custom PAM
    # Returns: a df of exon #, guides (23 bps), target (20 bps), fwd or rev, codon #, edit made
    
    # process mode
    if len(mode) == 3: 
        mode = base_editing_key[mode]
    assert len(mode) == 2
    
    # process PAM
    if PAM is None: 
        PAM = cas_key[cas_type]
    PAM_regex = process_PAM(PAM)
    
    # filter for PAM and contains editable base in window
    fwd_results = [g.copy() for g in gene_object.fwd_guides if PAM_regex.match(g[0][-len(PAM):]) and 
                                                    mode[0] in g[0][window[0]-1:window[1]]]
    for g in fwd_results: 
        # mutates all residues according to the mode
        original = g[0][1:12]
        mutated = g[0][1:window[0]-1] + g[0][window[0]-1:window[1]].replace(mode[0], 
                                                                            mode[1]) + g[0][window[1]:window[1]+4]
        assert(len(original)==len(mutated))
        # compares the residues to find which amino acids were altered and catalogs them
        start = (-1*(g[1]-1))+1
        original, mutated = original[start:start+9], mutated[start:start+9]
        original_aa, mutated_aa = '', ''
        edit, edit_ind = [], []
        for i in range(3): 
            if not original[i*3:(i+1)*3].isupper(): 
                continue
            original_aa += DNA_AA_map[original[i*3:(i+1)*3]]
            mutated_aa += DNA_AA_map[mutated[i*3:(i+1)*3]]
            if original_aa[-1] != mutated_aa[-1]: 
                edit.append(original_aa[-1] + ">" + mutated_aa[-1])
                edit_ind.append(int((g[2]+1+start+(i*3))/3))
                
        if len(edit) == 0: 
            edit.append('No Change')
        # append all information to dataframe
        g.append(original_aa)
        g.append(mutated_aa)
        g.append(edit)
        g.append(edit_ind)
        g.append('fwd')
        assert(len(g)) == 9
        
    # filter for PAM and contains editable base in window
    rev_results = [g.copy() for g in gene_object.rev_guides if PAM_regex.match(g[0][-len(PAM):]) and 
                                                    mode[0] in g[0][window[0]-1:window[1]]]
    for g in rev_results: 
        # mutates all residues according to the mode
        original = g[0][1:12]
        mutated = g[0][1:window[0]-1] + g[0][window[0]-1:window[1]].replace(mode[0], 
                                                                            mode[1]) + g[0][window[1]:window[1]+4]
        assert(len(original)==len(mutated))
        # compares the residues to find which amino acids were altered and catalogs them
        original = rev_complement(complements, original[g[1]:g[1]+9])
        mutated = rev_complement(complements, mutated[g[1]:g[1]+9])
        original_aa, mutated_aa = '', ''
        edit, edit_ind = [], []
        for i in range(3): 
            if not original[i*3:(i+1)*3].isupper(): 
                continue
            original_aa += DNA_AA_map[original[i*3:(i+1)*3]]
            mutated_aa += DNA_AA_map[mutated[i*3:(i+1)*3]]
            if original_aa[-1] != mutated_aa[-1]: 
                edit.append(original_aa[-1] + ">" + mutated_aa[-1])
                edit_ind.append(int((g[2]-5+(i*3))/3))
                
        if len(edit) == 0: 
            edit.append('No Change')
        # append all information to dataframe
        g.append(original_aa)
        g.append(mutated_aa)
        g.append(edit)
        g.append(edit_ind)
        g.append('rev')
        assert(len(g)) == 9
        
    results = fwd_results + rev_results
    return pd.DataFrame(results)

# function to change a PAM sequence into a regex sequence
def process_PAM(PAM): 
    PAM = PAM.replace("G", "[gG]{1}")
    PAM = PAM.replace("C", "[cC]{1}")
    PAM = PAM.replace("T", "[tT]{1}")
    PAM = PAM.replace("A", "[aA]{1}")
    PAM = PAM.replace("N", "[acgtACGT]{1}")
    return re.compile("({})".format(PAM))
    

In [15]:
# 1. generate a .fasta file of your gene using this protocol "Creating CRISPR sgRNA Libraries on CRISPOR.pdf"
# 2. inputs
#       filepath is your fasta file
#       output_dir can be any folder
#       n= length of your guide, 23 is standard
#       mode (ABE, CBE, or a list of edits like ["A", "C"])
#       cas_type (Sp, SpG)
#       target_codons=[] no targets default but you can look for specific amino acid residues you want to generate
#       window=[4,8] 4-8 inclusive is the default BE window but you can edit this
#       PAM=None default but you can input custom PAM (ex NGNG), if you leave as None the PAM is decided by cas_type
# 3. run the following code

AR = Gene_BaseEditing(filepath="230408_AR_Input.fasta", 
                      output_dir="results"
                     )
AR.find_all_guides(n=23)
find_gRNAs(gene_object= AR, 
           mode=        "CBE", 
           cas_type=    "Sp"
          ) #.to_csv('230409_AR_BESpCBE_library.csv') # if you want to save this dataframe

,0,1,2,3,4,5,6,7,8
0,AAGTGCAGTTAGGGCTGGGAAGG,1,4,0,VQL,V.L,[Q>.],[3],fwd
1,AGTGCAGTTAGGGCTGGGAAGGG,2,5,0,VQL,V.L,[Q>.],[3],fwd
2,CCGCCGTCCAAGACCTACCGAGG,0,39,0,PSK,LFK,"[P>L, S>F]","[14, 15]",fwd
3,GCGCGAAGTGATCCAGAACCCGG,2,89,0,REV,REV,[No Change],[],fwd
4,TGATCCAGAACCCGGGCCCCAGG,1,97,0,IQN,I.N,[Q>.],[34],fwd
...,...,...,...,...,...,...,...,...,...
374,ATGATCTCTGCCATCATTTCCGG,1,2698,7,AEI,AKI,[E>K],[898],rev
375,ACTTTCCCAGAAAGGATCTTGGG,1,2734,7,SGK,SKK,[G>K],[910],rev
376,GACTTTCCCAGAAAGGATCTTGG,2,2735,7,SGK,SEK,[G>E],[911],rev
377,TGGGCTTGACTTTCCCAGAAAGG,0,2742,7,VKP,VKP,[No Change],[],rev
